In [102]:
# Dependencies and Setup
import pandas as pd
import numpy as mp

# File to Load 
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data_df = pd.read_csv(school_data_to_load)
student_data_df = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete_df = pd.merge(student_data_df, school_data_df, how="left", on=["school_name", "school_name"])

#Set school as index for dataframes 
school_index = school_data_df.set_index('school_name') 
merged_index = school_data_complete_df.set_index('school_name')

## District Summary

In [103]:
# Calculate the total number of schools
unique_schools = school_data_complete_df["school_name"].value_counts().count()

# Calculate the total number of students
unique_students = school_data_complete_df["Student ID"].value_counts().count()

## Calculate the total budget
# Explore budget column; duplicate values exist; drop duplicates
remove_duplicates = school_data_complete_df.drop_duplicates(subset = 'budget')

# find sum of budget column with duplicates removed
total_budget = remove_duplicates["budget"].sum()

# Calculate the average math score 
avg_math = school_data_complete_df["math_score"].mean()

# Calculate the average reading score
avg_read = school_data_complete_df["reading_score"].mean()

# Calculate the percentage of students with a passing math score (70 or greater)
pass_math = school_data_complete_df.math_score[school_data_complete_df.math_score >= 70]
math_count = pass_math.count()
percent_math = math_count/unique_students * 100

# Calculate the percentage of students with a passing reading score (70 or greater)
pass_reading = school_data_complete_df.reading_score[school_data_complete_df.reading_score >= 70]
read_count = pass_reading.count()
percent_reading = read_count/unique_students * 100

# Calculate the percentage of students who passed math **and** reading (% Overall Passing)
pass_overall = pass_reading + pass_math
pass_count = pass_overall.count()
overall_percent = pass_count/unique_students * 100

# Create a dataframe to hold the above results
district_summary_df = pd.DataFrame({
    "Total Schools": [unique_schools],
    "Total Students": [unique_students],
    "Total Budget": [total_budget],
    "Avg Math Score": [avg_math],
    "Avg Reading Score": [avg_read],
    "% Passed Math": [percent_math],
    "% Passed Reading": [percent_reading],
    "% Passed Overall": [overall_percent],
})

# Format total budget column
district_summary_df["Total Budget"] = district_summary_df["Total Budget"].map("${:,.2f}".format)

# Set index as Total Schools to remove 0, and round numbers
district_summary_index = district_summary_df.set_index('Total Schools')
district_summary_index

,Total Students,Total Budget,Avg Math Score,Avg Reading Score,% Passed Math,% Passed Reading,% Passed Overall
Total Schools,,,,,,,
15,39170,"$24,649,428.00",78.985371,81.87784,74.980853,85.805463,65.172326


## School Summary

In [104]:
# Create df grouped by school name and school type
grouped_df = school_data_complete_df.groupby(["school_name"])

# Use count to find total students per school
students_per_school = grouped_df["Student ID"].count()

# Get school type
school_type = school_index['type']

# Find total budget
budget_per_school = school_index["budget"]

# Per Student Budget can be found by dividing total school budget by total number of students per school
budget_per_student = budget_per_school / students_per_school

# Use mean function to find average math and reading score
school_avg_math = grouped_df["math_score"].mean()
school_avg_read = grouped_df["reading_score"].mean()

# Calculate the percentage of students with a passing math score (70 or greater)
students_pass_math = student_data_df[student_data_df["math_score"] >= 70]
grouped_math = students_pass_math.groupby("school_name")
math_count = grouped_math["Student ID"].count()
school_passed_math = math_count/students_per_school *100

# Calculate the percentage of students with a passing reading score (70 or greater)
students_pass_reading = student_data_df[student_data_df['reading_score'] >= 70]
grouped_reading = students_pass_reading.groupby('school_name')
read_count = grouped_reading['Student ID'].count()
school_passed_reading = read_count/students_per_school *100

# Calculate the percentage of overall students who passed math and reading (70 or greater)
overall_passed = student_data_df[(student_data_df['math_score'] >= 70) 
                                 & (student_data_df['reading_score'] >= 70)]
grouped_overall = overall_passed.groupby('school_name')
overall_count = grouped_overall['Student ID'].count()
overall_per = overall_count/students_per_school *100

# Create a dataframe to hold the above results
school_summary_df = pd.DataFrame({
    'School Type': school_type,
    'Total Students': students_per_school,
    'Total School Budget': budget_per_school,
    'Per Student Budget': budget_per_student,
    'Average Math Score': school_avg_math,
    'Average Reading Score': school_avg_read,
    '% Passed Math': school_passed_math,
    '% Passed Reading': school_passed_reading,
    '% Passed Overall': overall_per,
})

# Format total budget and per student budget column
school_summary_df["Total School Budget"] = school_summary_df["Total School Budget"].map("${:,.2f}".format)
school_summary_df["Per Student Budget"] = school_summary_df["Per Student Budget"].map("${:,.2f}".format)

school_summary_df.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passed Math,% Passed Reading,% Passed Overall
school_name,,,,,,,,,
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,54.642283
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,54.289887
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455


## Top Performing Schools (By % Overall Passing)

In [105]:
top_performing_schools = school_summary_df.sort_values(['% Passed Overall'], ascending = False).reset_index()
top_performing_schools.head()

,school_name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passed Math,% Passed Reading,% Passed Overall
0,Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
1,Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.418349,83.848930,93.272171,97.308869,90.948012
2,Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
3,Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,90.582567
4,Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541


## Bottom Performing Schools (By % Overall Passing)

In [106]:
low_performing_schools = school_summary_df.sort_values(['% Passed Overall']).reset_index()
low_performing_schools.head()

,school_name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passed Math,% Passed Reading,% Passed Overall
0,Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,52.988247
1,Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
2,Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
3,Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
4,Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172


## Math Scores by Grade

In [107]:
# Create series for grades using student_data_df
ninth_grade = student_data_df.loc[student_data_df["grade"] == "9th"].groupby("school_name")
tenth_grade = student_data_df.loc[student_data_df["grade"] == "10th"].groupby("school_name")
eleventh_grade = student_data_df.loc[student_data_df["grade"] == "11th"].groupby("school_name")
twelfth_grade = student_data_df.loc[student_data_df["grade"] == "12th"].groupby("school_name")

# Find average of each group's math grades
ninth_m = ninth_grade["math_score"].mean()
tenth_m = tenth_grade["math_score"].mean()
eleventh_m = eleventh_grade["math_score"].mean()
twelfth_m = twelfth_grade["math_score"].mean()

# Combine the series into a dataframe
math_scores = pd.DataFrame({
            "9th": ninth_m,
            "10th": tenth_m,
            "11th": eleventh_m,
            "12th": twelfth_m,
})
math_scores.head()

,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164


## Reading Score by Grade 

In [108]:
# Find average of each group's reading grades
ninth_r = ninth_grade["reading_score"].mean()
tenth_r = tenth_grade["reading_score"].mean()
eleventh_r = eleventh_grade["reading_score"].mean()
twelfth_r = twelfth_grade["reading_score"].mean()

# Combine the series into a dataframe
reading_scores = pd.DataFrame({
            "9th": ninth_r,
            "10th": tenth_r,
            "11th": eleventh_r,
            "12th": twelfth_r,
})
reading_scores.head()

,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699


## Scores by School Spending

In [109]:
# Create bins and labels for bins to place values based upon Spending Ranges
bins = [0,585,615,645,675]
spending = ['<$585','$585-615','$615-645','$645-675']
# School_summary_df.info() tells us Per Student Budget is an object

# Clean up formatting to remove $ (object) from Per Student Budget column
school_summary_df['Per Student Budget'] = school_summary_df['Per Student Budget'].str.replace('$', '')

# Convert Per Student Budget column to float
school_summary_df['Per Student Budget'] = pd.to_numeric(school_summary_df['Per Student Budget'], 
                                                        errors = 'coerce')

# Slice data and place into bins
school_summary_df["Spending Ranges (Per Student)"] = pd.cut(school_summary_df["Per Student Budget"], 
                                                            bins, labels=spending)

# Create a GroupBy object based upon "Spending Group"
groupby_spending = school_summary_df.groupby("Spending Ranges (Per Student)")

# Get the average of each column within Groupby Spending
groupby_spending[["Average Math Score", "Average Reading Score", "% Passed Math", 
                               "% Passed Reading", "% Passed Overall"]].mean().round()

<ipython-input-109-25eb6e1d5bd5>:7: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  school_summary_df['Per Student Budget'] = school_summary_df['Per Student Budget'].str.replace('$', '')


,Average Math Score,Average Reading Score,% Passed Math,% Passed Reading,% Passed Overall
Spending Ranges (Per Student),,,,,
<$585,83.0,84.0,93.0,97.0,90.0
$585-615,84.0,84.0,94.0,96.0,90.0
$615-645,79.0,82.0,76.0,86.0,66.0
$645-675,77.0,81.0,66.0,81.0,54.0


## Scores by School Size

In [110]:
# Create bins
bins1 = [0, 1000, 2000, 5000]

# Create names for bins
size = ['Small (<1000)', 'Medium (1000 - 2000)', 'Large (2000 - 5000)']

# Slice data and place into bins
school_summary_df["School Size"] = pd.cut(school_summary_df["Total Students"],bins1, labels=size)

# Create a GroupBy object based upon "School Size"
groupby_size = school_summary_df.groupby("School Size")

# Get the average of each column within groupby_size
groupby_size[["Average Math Score", "Average Reading Score", "% Passed Math", 
                               "% Passed Reading", "% Passed Overall"]].mean().round()

,Average Math Score,Average Reading Score,% Passed Math,% Passed Reading,% Passed Overall
School Size,,,,,
Small (<1000),84.0,84.0,94.0,96.0,90.0
Medium (1000 - 2000),83.0,84.0,94.0,97.0,91.0
Large (2000 - 5000),78.0,81.0,70.0,83.0,58.0


## Scores by School Type

In [111]:
# Create a GroupBy object based upon "School Type"
groupby_type = school_summary_df.groupby("School Type")

# Get the average of each column within groupby_type
groupby_type[["Average Math Score", "Average Reading Score", "% Passed Math", 
                               "% Passed Reading", "% Passed Overall"]].mean().round()

,Average Math Score,Average Reading Score,% Passed Math,% Passed Reading,% Passed Overall
School Type,,,,,
Charter,83.0,84.0,94.0,97.0,90.0
District,77.0,81.0,67.0,81.0,54.0
